In [1]:
import clip
from glob import glob
import torch
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# import os
# import clip
# import torch
# from torchvision.datasets import CIFAR100

# # Load the model
# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load('ViT-B/32', device)

# # Download the dataset
# cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download=True, train=False)

# # Prepare the inputs
# image, class_id = cifar100[3637]
# image_input = preprocess(image).unsqueeze(0).to(device)
# text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)

# # Calculate features
# with torch.no_grad():
#     image_features = model.encode_image(image_input)
#     text_features = model.encode_text(text_inputs)

# # Pick the top 5 most similar labels for the image
# image_features /= image_features.norm(dim=-1, keepdim=True)
# text_features /= text_features.norm(dim=-1, keepdim=True)
# similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
# values, indices = similarity[0].topk(5)

# # Print the result
# print("\nTop predictions:\n")
# for value, index in zip(values, indices):
#     print(f"{cifar100.classes[index]:>16s}: {100 * value.item():.2f}%")

In [3]:
avail_models = [ 'RN50', 'RN50x64', 'ViT-B/32', 'ViT-L/14@336px' ]

model, preprocess = clip.load(avail_models[2], device)

In [4]:
dataset = "/home/mnjm/workspace/clip/dataset/onion/*/*/*.jpg"
imgs_l = glob(dataset)
imgs_l = [ x for x in imgs_l if "cropped" not in x ]
print(len(imgs_l))

7207


In [5]:
labels = ['raw', 'translucent', 'golden brown']
text_inputs = torch.cat([clip.tokenize(f"{label} chopped onions in a dark pan") for label in labels]).to(device)

with torch.no_grad():
    text_embedds = model.encode_text(text_inputs)
text_embedds /= text_embedds.norm(dim=-1, keepdim=True)
print(text_embedds)

tensor([[-0.0834,  0.0474,  0.0749,  ...,  0.0463, -0.0332,  0.0344],
        [-0.0727,  0.0376,  0.0495,  ...,  0.0397, -0.0219,  0.0050],
        [-0.0446,  0.0228,  0.0587,  ...,  0.0628, -0.0233,  0.0036]])


In [6]:
t, c = 0, 0
true_lbl_l, pred_lbl_l = [], []
for img_p in imgs_l:
    img = Image.open(img_p).convert("RGB")
    img_cv = cv2.imread(img_p)
    img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
    if img_cv.shape[0] == 128 and img_cv.shape[1] == 128: continue
    t += 1
    img = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        img_embedd = model.encode_image(img)
    img_embedd /= img_embedd.norm(dim=-1, keepdim=True)
    similarity = (img_embedd @ text_embedds.T).softmax(dim=-1)
    similarity = np.array(similarity).reshape(-1)
    pred_lbl = np.argmax(similarity)
    score = np.max(similarity) * 100.0
    true_lbl = int(img_p.split(os.path.sep)[-2])
    if true_lbl != pred_lbl:
        c += 1
        # plt.title(f"{labels[pred_lbl]}({score:.2f}) (True: {labels[true_lbl]})")
        # plt.imshow(img_cv)
        # plt.show()

In [7]:
from sklearn.metrics import confusion_matrix
print(f"{c}/{t}")
assert len(true_lbl_l) == len(pred_lbl_l)
confusion_matrix(true_lbl_l, pred_lbl_l)

3782/7131


array([], shape=(0, 0), dtype=int64)